In [1]:
# Importing the necessary libraries

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import os
import time
import copy
import pandas as pd
from PIL import Image
from os import listdir
from sklearn.metrics import confusion_matrix
from torchvision import datasets, models, transforms

In [2]:
# Defining necessary file paths

model_path = '../Model/Pesticides_Prescription_Model.sav'
labels_path = '../Model/Class_Labels_Pesticides_Prescription_Model.csv'
test_data_dir = '../Dataset/test'

In [3]:
# Restoring the model

def initialize_alexnet(model_name, num_classes, feature_extract=True, use_pretrained=True):

  model_ft = None
  input_size = 0

  if model_name == "alexnet":
    model_ft = models.alexnet(pretrained=use_pretrained)

    if feature_extract:
        for param in model_ft.parameters():
            param.requires_grad = False
    
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)

    input_size = 224
  else:
        print("Model name is invalid!")
        exit()

  return model_ft,input_size

model, _ = initialize_alexnet("alexnet", num_classes=15)
model.load_state_dict(torch.load(model_path))
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
# Reading the labels file

labels_df = pd.read_csv(labels_path, header=None, names=['idx','Label'])
labels_dict = dict(zip(labels_df.idx, labels_df.Label))

In [5]:
# Predicting the output

def predict(img):
    transform = transforms.Compose([            
    transforms.Resize(256),                    
    transforms.CenterCrop(224),                
    transforms.ToTensor(),                     
    transforms.Normalize(                      
    mean=[0.485, 0.456, 0.406],                
    std=[0.229, 0.224, 0.225]                  
    )])

    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)  
    out = model(batch_t)
    _, index = torch.max(out, 1)
    
    return index

def get_label(labels_dict, idx=-1):
    if (idx>=0 and idx<=14):
        label = labels_dict[idx]
    else:
        label = ''
    return label

In [6]:
# Getting actual and predicted values for the test dataset

actual_label_list = []
predicted_lebel_list = []
list_test_data_dir = listdir(test_data_dir)
for leaf_image_dir in list_test_data_dir :
        image_list = listdir(f"{test_data_dir}/{leaf_image_dir}")
        for image in image_list:
          image_file = f"{test_data_dir}/{leaf_image_dir}/{image}"
          if image_file.endswith(".jpg") == True or image_file.endswith(".JPG") == True:
            actual_label_list.append(leaf_image_dir)
            img = Image.open(image_file)
            pred_idx = predict(img)
            predicted_lebel = get_label(labels_dict, pred_idx.numpy()[0])
            predicted_lebel_list.append(predicted_lebel)

In [7]:
# Confusion matrix

cm = confusion_matrix(actual_label_list, predicted_lebel_list)
cm

array([[ 99,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1, 148,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  4,   0,  94,   0,   0,   0,   0,   1,   0,   0,   0,   1,   0,
          0,   0],
       [  0,   0,   0,  14,   2,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1,   0,   0,   2,  92,   1,   0,   0,   3,   0,   0,   0,   0,
          1,   0],
       [  0,   0,   0,   0,   0, 210,   0,   1,   1,   0,   0,   1,   1,
          0,   0],
       [  2,   0,   0,   0,   2,  16,  52,   0,   6,   1,   0,  15,   3,
          2,   1],
       [  0,   0,   0,   0,   0,   1,   0, 158,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   8,   3,   0,   4, 165,   1,   0,   5,   1,
          4,   1],
       [  1,   0,   0,   0,   0,   2,   0,   1,   2,  78,   0,   6,   0,
          2,   4],
       [  0,   0,   0,   0,   0,   0,   0,   1,   0,   0,  37,   0,   0,
       

In [8]:
# Accuracy

accuracy = (cm.trace()/cm.sum())
accuracy

0.8978102189781022

In [9]:
# Precission

precision = np.diag(cm) / np.sum(cm, axis = 0)
precision

array([0.81147541, 0.98666667, 1.        , 0.875     , 0.87619048,
       0.88983051, 1.        , 0.82722513, 0.93220339, 0.96296296,
       0.925     , 0.82038835, 0.84677419, 0.9025974 , 0.95294118])

In [10]:
# Macro average precision

macro_average_precision = np.mean(precision)
macro_average_precision

0.9072837111321501

In [11]:
# Recall

recall = np.diag(cm) / np.sum(cm, axis = 1)
recall

array([0.98019802, 0.99328859, 0.94      , 0.875     , 0.92      ,
       0.98130841, 0.52      , 0.99371069, 0.859375  , 0.8125    ,
       0.97368421, 0.9494382 , 0.74468085, 0.82248521, 0.98181818])

In [12]:
# Macro average recall

macro_average_recall = np.mean(recall)
macro_average_recall

0.8898324910800646

In [13]:
# F1 score

F1 = 2 * (precision * recall) / (precision + recall)
F1

array([0.88789238, 0.98996656, 0.96907216, 0.875     , 0.89756098,
       0.93333333, 0.68421053, 0.90285714, 0.89430894, 0.88135593,
       0.94871795, 0.88020833, 0.79245283, 0.86068111, 0.96716418])

In [14]:
# # Macro average F1 score

macro_average_F1 = np.mean(F1)
macro_average_F1

0.890985490407892

In [15]:
from sklearn.metrics import classification_report
print(classification_report(actual_label_list, predicted_lebel_list))

                                 precision    recall  f1-score   support

    Bell_pepper__Bacterial_spot       0.81      0.98      0.89       101
           Bell_pepper__Healthy       0.99      0.99      0.99       149
           Potato__Early_blight       1.00      0.94      0.97       100
                Potato__Healthy       0.88      0.88      0.88        16
            Potato__Late_blight       0.88      0.92      0.90       100
         Tomato__Bacterial_spot       0.89      0.98      0.93       214
           Tomato__Early_blight       1.00      0.52      0.68       100
                Tomato__Healthy       0.83      0.99      0.90       159
            Tomato__Late_blight       0.93      0.86      0.89       192
              Tomato__Leaf_mold       0.96      0.81      0.88        96
           Tomato__Mosaic_virus       0.93      0.97      0.95        38
     Tomato__Septoria_leaf_spot       0.82      0.95      0.88       178
            Tomato__Target_spot       0.85      0.